In [1]:
# Check if i can still read in a dat file
# I think this one is the real measured MSS
import pathlib

# Using this one for now, since it's the "real" one that Laura applied the denoising to
filepath = pathlib.Path(
    "/home/mh19137/geog_rdsf/data/projects/SING/richard_stuff/Table2/currents/dtu18_eigen-6c4_do0280_rr0004_cs.dat"
)
assert filepath.exists()

In [ ]:
from current_denoising.generation import ioutils

data = ioutils.read_currents(filepath)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.hist(data.flat)
for k, v in list(dict(zip(*np.unique(data.flat, return_counts=True))).items())[:5]:
    print(k, v)

In [ ]:
data[data == 0] = np.nan
print(np.nanmin(data), np.nanmax(data))

In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(12, 6))

im = axis.imshow(data, origin="lower", norm="log")
axis.set_title("Data 1")

fig.colorbar(im)

plt.tight_layout()

In [ ]:
from current_denoising.generation.ioutils import extract_tiles

rng = np.random.default_rng(1234)

tiles = extract_tiles(rng, data, num_tiles=16, max_rms=np.inf, max_latitude=10.0, tile_size=32)

In [ ]:
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
for axis, tile in zip(axes.flat, tiles):
    im = axis.imshow(tile, origin="lower", norm="log", vmin=np.nanmin(data), vmax=np.nanmax(data))